In [1]:
import sys
sys.path.append('..')

In [2]:
!killall tensorboard
import os
os.system('tensorboard --logdir=/tmp/tboard --port=7007 &')

tensorboard: no process killed


0

In [3]:
import tensorflow as tf
from keras_utils import reset_tf_session
s=reset_tf_session()

ModuleNotFoundError: No module named 'keras_utils'

Using TensorFlow backend.
